# Another take on the segment requirements

We will try going the other way, using Laurent P's approach to get $\mu$, use that to. build $C_y$ and go the other way around. We calculate $C_x$ from $C_y$ and look at that $C_x$ matrix, see how diagonal it is or not. We can truncate $C_x$ to the diagonal terms and run a Monte Carlo with these dependent modes and see if we get to the requested target contrast for which we drew the $\mu$ in the first place.

1. set target contrat in code cell 2 (e.g. `1e-10`)
2. set apodizer design in code cell 3 (e.g. `small`)
3. comment in correct data path in code cell 3 (e.g. `[...]/2020-01-27T23-57-00_luvoir-small`)

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
from astropy.io import fits
import astropy.units as u
import hcipy as hc
from hcipy.optics.segmented_mirror import SegmentedMirror

os.chdir('../../pastis/')
import util_pastis as util
from e2e_simulators.luvoir_imaging import LuvoirAPLC

In [ ]:
c_target = 1e-10
nmodes = 120

## Instantiate LUVOIR telescope for full functionality

In [ ]:
apodizer_design = 'small'

savedpath = '/Users/ilaginja/Documents/data_from_repos/pastis_data/2020-01-27T23-57-00_luvoir-small'
#savedpath = '/Users/ilaginja/Documents/data_from_repos/pastis_data/2020-01-28T02-17-18_luvoir-medium'
#savedpath = '/Users/ilaginja/Documents/data_from_repos/pastis_data/2020-01-28T04-45-55_luvoir-large'

In [ ]:
# Instantiate LUVOIR
sampling = 4
# This path is specific to the paths used in the LuvoirAPLC class
optics_input = '/Users/ilaginja/Documents/LabWork/ultra/LUVOIR_delivery_May2019/'

luvoir = LuvoirAPLC(optics_input, apodizer_design, sampling)

In [ ]:
# Make reference image
luvoir.flatten()
psf_unaber, ref = luvoir.calc_psf(ref=True)
norm = ref.max()

In [ ]:
# Make dark hole
dh_outer = hc.circular_aperture(2*luvoir.apod_dict[apodizer_design]['owa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_inner = hc.circular_aperture(2*luvoir.apod_dict[apodizer_design]['iwa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_mask = (dh_outer - dh_inner).astype('bool')

plt.figure(figsize=(18, 6))
plt.subplot(131)
hc.imshow_field(psf_unaber/norm, norm=LogNorm())
plt.subplot(132)
hc.imshow_field(dh_mask)
plt.subplot(133)
hc.imshow_field(psf_unaber/norm, norm=LogNorm(), mask=dh_mask)

In [ ]:
dh_intensity = psf_unaber/norm * dh_mask
baseline_contrast = util.dh_mean(dh_intensity, dh_mask)
#np.mean(dh_intensity[np.where(dh_intensity != 0)])
print('Baseline contrast:', baseline_contrast)

## Read PASTIS matrix and mode matrix

In [ ]:
# Load PASTIS modes - piston value per segment per mode
pastismodes = np.loadtxt(os.path.join(savedpath, 'results', 'singular_modes.txt'))
print('pastismodes.shape: {}'.format(pastismodes.shape))
# pastismodes[segs, modes]

# Load PASTIS matrix
pastismatrix = fits.getdata(os.path.join(savedpath, 'matrix_numerical', 'PASTISmatrix_num_piston_Noll1.fits'))

In [ ]:
# Calculate the inverse of the PASTIS mode matrix
# This is ModeToSegs in Mathematica
modestosegs = np.linalg.pinv(pastismodes)

## Calculate the $\mu$ map

In [ ]:
# Now calculate all mean contrasts of the pastis modes directly (as-is, with natural normalization)
c_avg = []
for i in range(nmodes):
    c_avg.append(util.pastis_contrast(pastismodes[:,i]*u.nm, pastismatrix) + baseline_contrast)
    
print(c_avg)

In [ ]:
# Calculate segment requirements
mus = np.sqrt(((c_target-baseline_contrast)/nmodes)/(np.dot(c_avg-baseline_contrast, np.square(modestosegs))))
print(mus)

## Construct $C_y$ by using the $\mu$ values as standard deviation

In [ ]:
cy = np.diag(np.square(mus))
print(cy)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(cy)
plt.title('$C_y$')
plt.xlabel('segments')
plt.ylabel('segments')
plt.colorbar()

Check that the mode matrix $U$ (= `pastismodes`) is unitary. `modestosegs` (= $U^{-1}$) is the inverse of `pastismodes` (= $U$).

Means, check that $U^T = U^{-1}$

In [ ]:
np.allclose(np.transpose(pastismodes), modestosegs)

This means $U$ is unitary.

We know that $y = U \cdot x$ and therefore:

$$y = U \cdot x$$
$$C_x = E(x \cdot x^T)$$
$$C_y = E(y \cdot y^T) = U \cdot E(x \cdot x^T) \cdot U^T$$
$$C_y = U \cdot C_x \cdot U^T$$

Go the other way around:

$$x = U^{-1} \cdot y$$
$$C_x = U^{-1} \cdot C_y \cdot (U^{-1})^T$$
$$C_x = U^{-1} \cdot C_y \cdot U$$
$$C_x = U^{T} \cdot C_y \cdot U$$

Now calculate $C_x$ with this.

In [ ]:
cx = np.dot(np.transpose(pastismodes), np.dot(cy, pastismodes))

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(cx)
plt.title('$C_x$')
plt.xlabel('modes')
plt.ylabel('modes')
plt.colorbar()

In [ ]:
cx_diag = np.sqrt(np.diag(cx))

In [ ]:
plt.plot(cx_diag)

We have two bases: modes and segments. Both are equivalent because they are just a change of base.
Because there is a linear transform between both, there is a covariance matrix relationship.
I you assume non-correlation in one space, there will be correlation in the other space.

It's very assymmetric in the sense that when the modes are uncorrelated, the resulting segment distribution is very correlated, see notebook 14/15. Whereas when the segments are uncorrelated, the modes are not quite as correlated as the other way around.

This makes sense since already one mode by itself imposes a correlation between all segments (because they have to build that mode).

In [ ]:
cy_backwards = np.dot(pastismodes, np.dot(np.diag(np.diag(cx)), np.transpose(pastismodes)))

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(cy_backwards)
plt.title('cy_backwards')
plt.xlabel('segment')
plt.ylabel('segment')
plt.colorbar()

Use the diagonal from $C_x$ to build an equivalent of the cumulative target contrast curve, now with this "optimized" error budget.

In [ ]:
# Calculate cumulative contrast
cont_cum_pastis = []
for maxmode in range(nmodes):
    aber = np.nansum(pastismodes[:, :maxmode+1]*cx_diag[:maxmode+1], axis=1)
    aber *= u.nm
    
    contrast_matrix = util.pastis_contrast(aber, pastismatrix) + baseline_contrast
    cont_cum_pastis.append(contrast_matrix)

In [ ]:
plt.figure(figsize=(16, 10))
plt.plot(cont_cum_pastis, label='PASTIS matrix')
plt.title('Cumulative contrast for target $C$ = ' + str(c_target), size=15)
plt.xlabel('Mode number', size=15)
plt.ylabel('Constrast', size=15)

In [ ]:
cont_cum_pastis[-1]

## Calculate statistical mean contrast

We derived

$$\left< c \right> = tr(C_y M) + c_0,$$

where $M$ is the PASTIS matrix, $C_y$ the covariance matrix in segment-space and $c_0$ the coronagraph floor. We want to confirm this.

Our $C_y$ that we calculated in this notebook is specifically for the target contrast $c_{target}$, so that's what we want to recover.

In [ ]:
print('c_target = {}'.format(c_target))

In [ ]:
mean_c_stat = np.trace(np.matmul(cy, pastismatrix)) + baseline_contrast
print('Statistical mean contrast = {}'.format(mean_c_stat))